# Space Images Classifier - Using Kaggle dataset

https://www.kaggle.com/datasets/abhikalpsrivastava15/space-images-category?utm_source=chatgpt.com

### Notebook 3 - Building Deep Learning Model

# Import librairies

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sys
import os
import json
# Add the root folder to Python's module search path
sys.path.append(os.path.abspath(os.path.join(".."))) 
# Import the project configuration
from config import DEVICE, ORIGINAL_DATA_PATH, OUTPUT_PATH, IMG_SIZE, BATCH_SIZE, NUM_WORKERS, SEED, TRAIN_RATIO, VAL_RATIO, TEST_RATIO

import shutil
from pathlib import Path
import cv2
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.transforms.functional as TF

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.utils import class_weight

# Building model (EfficientNet-B0)

https://medium.com/image-processing-with-python/efficientnetb0-architecture-stem-layer-496c7911a62d

https://medium.com/@kdk199604/efficientnet-smarter-not-just-bigger-neural-networks-94db3e2f8699



In [ ]:
class SpaceClassifier(nn.Module):
    """EfficientNet-B0 based classifier for space images"""
    
    def __init__(self, num_classes, pretrained=True):
        super(SpaceClassifier, self).__init__()
        
        # Load pre-trained EfficientNet-B0
        self.backbone = models.efficientnet_b0(pretrained=pretrained)
        
        # Freeze backbone during the first stage to speed up the training (avoid replacing pretrained features)
        for param in self.backbone.parameters():
            param.requires_grad = False
        
        # Get the number of features output by the backbone before the original classifier
        in_features = self.backbone.classifier[1].in_features
        # Replace classifier default head
        # Progressive reduction feature dimensionality (from 1280 default value to num_classes) and it helps for overfitting
        # Introduction of non-linearity function between fully connected layers
        # Use of higher dropout early and smaller later to help regularization of deeper layers and prevent overfitting as well
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(p=0.4, inplace=True),
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3, inplace=True),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.backbone(x)
    
    # Unfreeze backbone for fine-tuning
    def unfreeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = True

In [4]:
try:
    with open("config_dynamic.json") as f:
        dynamic_config = json.load(f)
except FileNotFoundError:
    dynamic_config = {}
    
NUM_CLASSES = dynamic_config.get("NUM_CLASSES", 0)
class_names = dynamic_config.get("class_names", [])

# Create model
model = SpaceClassifier(NUM_CLASSES, pretrained=True)
model = model.to(DEVICE)

print("=" * 80)
print(f"Model created: EfficientNet-B0")
print(f"Device: {DEVICE}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

print("=" * 80)

Model created: EfficientNet-B0
Device: mps
Total parameters: 4,796,290
Trainable parameters: 788,742


In [6]:
print(model.backbone.classifier)

Sequential(
  (0): Dropout(p=0.4, inplace=True)
  (1): Linear(in_features=1280, out_features=512, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.3, inplace=True)
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): ReLU(inplace=True)
  (6): Dropout(p=0.2, inplace=True)
  (7): Linear(in_features=256, out_features=6, bias=True)
)


# Training utilities

# End of notebook 3 - Building Deep Learning model